## 0. Установка путей

In [4]:
SCRIPT_PATH = "Tensorflow/scripts/generate_tfrecord.py"
WORKSPACE_PATH = "Tensorflow/workspace"
APIMODEL_PATH = "Tensorflow/models"
LABEL_MAP_PATH = WORKSPACE_PATH + "/annotations/label_map.pbtxt"
TRAIN_RECORD_PATH = WORKSPACE_PATH + "/annotations/train.record"
TEST_RECORD_PATH = WORKSPACE_PATH + "/annotations/test.record"
MODEL_PATH = WORKSPACE_PATH + "/models"
TRAIN_PATH = WORKSPACE_PATH + "/images/train"
TEST_PATH = WORKSPACE_PATH + "/images/test"
CHECKPOINT_PATH = MODEL_PATH + "/my_ssd_mobnet/"
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + "/pre-trained-models"
PRETRAINED_MODEL_CONFIG = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'
CUSTOM_MODEL_NAME = "my_ssd_mobnet"
APP_TITLE = "Обнаружение маски"

## 1. Создание карты меток

In [15]:
labels = [{"name": "Mask", "id": 1}, {"name": "NoMask", "id": 2}]

with open(LABEL_MAP_PATH, "w") as f:
    for label in labels:
        f.write("item {\n")
        f.write("\tname:'{}'\n".format(label["name"]))
        f.write("\tid:{}\n".format(label["id"]))
        f.write("}\n")

## 2. Создание файлов-записей обучения и теста Tensorflow

In [16]:
!python {SCRIPT_PATH} -x {TRAIN_PATH} -l {LABEL_MAP_PATH} -o {TRAIN_RECORD_PATH}
!python {SCRIPT_PATH} -x {TEST_PATH} -l {LABEL_MAP_PATH} -o {TEST_RECORD_PATH}

Файл TFRecord успешно создан: Tensorflow/workspace/annotations/train.record
Файл TFRecord успешно создан: Tensorflow/workspace/annotations/test.record


## 3. Загрузка предварительно обученных моделей TensorFlow

In [17]:
!cd Tensorflow

Cloning into 'models'...
Updating files:  77% (2259/2900)
Updating files:  78% (2262/2900)
Updating files:  79% (2291/2900)
Updating files:  80% (2320/2900)
Updating files:  81% (2349/2900)
Updating files:  82% (2378/2900)
Updating files:  83% (2407/2900)
Updating files:  84% (2436/2900)
Updating files:  85% (2465/2900)
Updating files:  86% (2494/2900)
Updating files:  87% (2523/2900)
Updating files:  88% (2552/2900)
Updating files:  89% (2581/2900)
Updating files:  90% (2610/2900)
Updating files:  91% (2639/2900)
Updating files:  92% (2668/2900)
Updating files:  93% (2697/2900)
Updating files:  94% (2726/2900)
Updating files:  95% (2755/2900)
Updating files:  96% (2784/2900)
Updating files:  97% (2813/2900)
Updating files:  98% (2842/2900)
Updating files:  99% (2871/2900)
Updating files: 100% (2900/2900)
Updating files: 100% (2900/2900), done.


In [ ]:
# !git clone https://github.com/tensorflow/models

## 4. Загрузка и распаковка одноэтапной модели обнаружения объектов SSD-MobileNet V2

In [18]:
# # Загрузка тренированной модели
# wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
# # Перемещение архива
# !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
# # Перемещение в папку с архивом
# !cd {PRETRAINED_MODEL_PATH}
# # Распакова архива
# !tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## 5. Копирование конфигурации модели в папку для обучения

In [19]:
!mkdir {'Tensorflow\workspace\models\\' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_CONFIG} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


## 6. Обновления конфигурации для трансферного обучения

In [1]:
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
from object_detection.utils import config_util, label_map_util

In [5]:
CONFIG_FILE = MODEL_PATH + "/" + CUSTOM_MODEL_NAME + "/pipeline.config"
config = config_util.get_configs_from_pipeline_file(CONFIG_FILE)

In [22]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_FILE, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [23]:
# Список изменений в конфигурации
pipeline_config.model.ssd.num_classes = 2 # Количество классов (В маске и без маски)
pipeline_config.train_config.batch_size = 6 # Чем больше размер партии - тем больше точность модели и длительность выполнение тренировки
pipeline_config.train_config.fine_tune_checkpoint = (PRETRAINED_MODEL_PATH + "/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0")
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = LABEL_MAP_PATH
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [TRAIN_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = LABEL_MAP_PATH
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [TEST_RECORD_PATH]

In [24]:
# Запись изменений в файл конфигурации
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(CONFIG_FILE, "wb") as f:
    f.write(config_text)

## 7. Тренировка модели

In [25]:
print(
    """python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=6000"""
    .format(APIMODEL_PATH, MODEL_PATH, CUSTOM_MODEL_NAME, MODEL_PATH, CUSTOM_MODEL_NAME
    )
)
# Для тренировки модели необходимо запустить выведенную команду в корневой директории проекта и только после завершения тренировки продолжить запуск кода

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=6000


## 8. Загрузка тренировочной модели с чекпоинта

In [2]:
from object_detection.builders import model_builder
from object_detection.utils import visualization_utils as viz_utilz
import os

In [9]:
# Загрузка конфигурации и построение модели обнаружения
configs = config_util.get_configs_from_pipeline_file(CONFIG_FILE)
detection_model = model_builder.build(model_config=configs["model"], is_training=False)

# Восстановление точки сохранения
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, "ckpt-2")).expect_partial()

In [10]:
@tf.function
def detect(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 9. Обнаружение маски в реальном времени

In [11]:
import cv2  # Импорт библиотеки OpenCV для компьютерного видения
import numpy as np

In [12]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)

In [13]:
camera = cv2.VideoCapture(0) # Видеозахват веб-камеры. В качестве аргумента указывается ID-устройства, либо путь к медиафайлу
# # Получение ширины и высоты окна
# width, height = ( 
#     int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
#     int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)),
# )

while True:
    ret, frame = camera.read() # Считывание кадров с камеры
    image_arr = np.array(frame) # Перевод кадров в массив

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_arr, 0), dtype=tf.float32)
    detections = detect(input_tensor)

    num_detections = int(detections.pop("num_detections"))
    detections = {
        key: value[0, :num_detections].numpy() for key, value in detections.items()
    }
    detections["num_detections"] = num_detections
    detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

    label_id_offset = 1 # Для отсчёта классов с единицы, вместо нуля
    image_arr_with_detections = image_arr.copy() # Копирование оригинального кадра для наложения визуализации

    # Визуализация
    viz_utilz.visualize_boxes_and_labels_on_image_array(
        image_arr_with_detections,
        detections["detection_boxes"],
        detections["detection_classes"] + label_id_offset,
        detections["detection_scores"],
        category_index,
        use_normalized_coordinates=True, # Установка визуализации поверх объекта, а не в координатах (0, 0)
        max_boxes_to_draw=2, # Максимальное количество визуализированных квадратов
        line_thickness=2, # Толщина линий квадрата
        min_score_thresh=0.5, # Минимальная отметка точности - 50%
        agnostic_mode=False,
    )

    cv2.imshow(APP_TITLE, cv2.resize(image_arr_with_detections, (1000, 750)))  # Трансляция с камеры и установка размера окна

    if cv2.waitKey(1) == ord("q"): # Привязка завершения обнаружения на клавишу 'q'
        break
    
camera.release() # Высвобождение камеры
cv2.destroyAllWindows() # Закрытие окон OpenCV